<a href="https://colab.research.google.com/github/sedmonster/EmpathoBot/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


user_rating_counts = ratings['user_id'].value_counts()
ratings = ratings[ratings['user_id'].isin(user_rating_counts[user_rating_counts >= 200].index)]

book_rating_counts = ratings['book_id'].value_counts()
ratings = ratings[ratings['book_id'].isin(book_rating_counts[book_rating_counts >= 100].index)]

df = ratings.merge(books, on='book_id')

pivot = df.pivot_table(index='title', columns='user_id', values='rating').fillna(0)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(pivot.values)

def get_recommends(book_title):

    if book_title not in pivot.index:
        return f"Book '{book_title}' not found in dataset."


    book_idx = list(pivot.index).index(book_title)


    distances, indices = model.kneighbors(
        pivot.iloc[book_idx, :].values.reshape(1, -1),
        n_neighbors=6
    )


    recommendations = []
    for i in range(1, len(distances[0])):
        title = pivot.index[indices[0][i]]
        distance = distances[0][i]
        recommendations.append([title, distance])


    recommendations.reverse()

    return [book_title, recommendations]


test_result = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(test_result)